# Никитин Дмитрий 4217
## Вариант №3
"Классификация текстов (IMDb Reviews)"
Описание задачи:
Используем датасет IMDb Reviews, который состоит из текстовых отзывов о фильмах, помеченных как положительные (positive) или отрицательные (negative). Нужно обучить нейронную сеть для классификации текстов на основе этих данных.
Шаги работы:
1. *Загрузка данных:* датасет IMDb Reviews доступен через Keras и на Kaggle.
2. *Предобработка данных:* тексты в датасете представлены в виде индексов слов в словаре. Надо преобразовать их обратно в текстовый формат для дальнейшей обработки. Нужно преобразовать индексы в массивы с использованием pad_sequences, чтобы тексты были одинаковой длины
3. *Обучение модели:* создать нейронную сеть для классификации текста на положительный или отрицательный отзыв.
4. *Тестирование:* протестировать модель на тестовых данных. Оценить точность модели на тестовой выборке.

Сначала был проведён просмотр датасета.

In [16]:
import pandas as pd

dataset = pd.read_csv("data/IMDB Dataset.csv")

dataset.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


Далее был очищен текст (удалены лишние символы, приведён к нижнему регистру).

In [17]:
import re

# Очистка текста
def clean_text(text: str) -> str:
  # Удаление тегов HTML
  text: str = re.sub(r'<.*?>', '', text)

  # Удаление символов, не являющихся буквой, цифрой, нижним подчёркиванием или
  # пробелом
  text: str = re.sub(r'[^\w\s]', '', text)

  text: str = text.lower()

  return text

# Превращение positive и negative оценок в 1 и 0 соответственно
def sentiment_to_number(sentiment: str) -> int:
  if sentiment == "positive":
    return 1
  else:
    return 0

dataset['sentiment'] = dataset['sentiment'].apply(sentiment_to_number)
dataset['review'] = dataset['review'].apply(clean_text)

dataset


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1
...,...,...
49995,i thought this movie did a down right good job...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,i am a catholic taught in parochial elementary...,0
49998,im going to have to disagree with the previous...,0


Далее текст был преобразован в числовой формат или же токенизирован. Токенизация — это процесс преобразования текста в последовательность чисел (токенов), которые могут быть обработаны нейронной сетью. В данном случае используется Tokenizer из библиотеки TensorFlow/Keras, который создаёт словарь из наиболее часто встречающихся слов и присваивает каждому слову уникальный индекс. Слова, не вошедшие в словарь, заменяются на специальный токен <OOV> (Out Of Vocabulary). После этого тексты преобразуются в последовательности чисел, а затем дополняются нулями или обрезаются до одинаковой длины с помощью функции pad_sequences, чтобы все входные данные имели единый размер для обработки моделью.

In [18]:
import tensorflow as tf

vocab_size = 10000 # Размер словаря
max_length = 200 # Максимальный размер последовательности

# Создание токенизатора - редкоиспользуемые слова были помечены как <00V>
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<00V>")
# Обучение токенизатора
tokenizer.fit_on_texts(dataset['review'])

# Преобразование текста в последовательности чисел
sequences = tokenizer.texts_to_sequences(dataset['review'])

# Обрезанные и продлённые последовательности одной длины
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

Далее данные были разделены на обучающую и тестовую выборки.

In [19]:
from sklearn.model_selection import train_test_split

# 80% датасета используется для тренировки, 20% - для тестирования
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, dataset['sentiment'], test_size=0.2, random_state=0)

Создание нейронной сети начинается с определения её архитектуры с помощью tf.keras.Sequential, которая позволяет последовательно добавлять слои. Первый слой — Embedding, преобразует целочисленные индексы слов в плотные векторы фиксированной размерности (в данном случае 64), что позволяет модели работать с семантическим значением слов. Далее идёт слой Flatten, который "выравнивает" многомерные данные в одномерный вектор, подготавливая их для полносвязных слоёв. Затем добавляется полносвязный слой Dense с 64 нейронами и функцией активации ReLU f(x) = max(0, x), который помогает модели выявлять сложные зависимости в данных. Выходной слой Dense с одним нейроном и функцией активации sigmoid, преобразующей выход в вероятность: f(x) = 1 / (1 + exp(-x)), используется для бинарной классификации, выдавая вероятность принадлежности к классу 1 (например, положительный отзыв). После создания архитектуры модель компилируется с использованием оптимизатора Adam, функции потерь binary_crossentropy (для бинарной классификации) и метрики accuracy для оценки точности. Метод model.summary() выводит структуру модели, включая информацию о каждом слое, форме выходных данных и количестве обучаемых параметров.

In [20]:
# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length, input_shape=(max_length,)), # Слой Embedding
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Выходной слой для бинарной классификации
])

# Компиляция модели
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

# Вывод структуры модели
print(model.summary())

C:\Users\johnworker\.conda\envs\Lab1\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\Users\johnworker\.conda\envs\Lab1\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 200, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       819,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,459,329 (5.57 MB)

 Trainable params: 1,459,329 (5.57 MB)

 Non-trainable params: 0 (0.00 B)

None


Вывод говорит о том, что модель была создана верно. Отображены все 4 слоя, заданные в коде.

Слой embedding преобразует целочисленные индексы слов в плотные векторы фиксированной размерности. В данном случае каждое слово является вектором из 64 чисел.

Слой flatten преобразует её в вектор длины 200 * 64 = 12,800.

Верхний dense - полносвязный слой, который обучается на данных. Каждый нейрон в этом слое принимает входные данные от всех элементов предыдущего слоя и применяет к ним веса и смещения.

Вход: вектор длины 12,800.

Выход: вектор длины 64.

Функция активации relu (Rectified Linear Unit) добавляет нелинейность: f(x) = max(0, x).

Нижний dense - выходной слой для бинарной классификации.

Выдаёт вероятность принадлежности к классу 1.

Вход: вектор длины 64.

Выход: одно число (от 0 до 1).

Функция активации sigmoid преобразует выход в вероятность: f(x) = 1 / (1 + exp(-x)).

Далее было проведено обучение модели на данных для обучения.

In [21]:
# Обучение модели
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=512,
    validation_data=(X_test, y_test)
)

Epoch 1/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6180 - loss: 0.6243 - val_accuracy: 0.8451 - val_loss: 0.3564
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8945 - loss: 0.2622 - val_accuracy: 0.8584 - val_loss: 0.3359
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.9624 - loss: 0.1276 - val_accuracy: 0.8445 - val_loss: 0.3781
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9950 - loss: 0.0386 - val_accuracy: 0.8492 - val_loss: 0.4179
Epoch 5/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9986 - loss: 0.0154 - val_accuracy: 0.8485 - val_loss: 0.4535


Во время выполнения команды model.fit происходит процесс обучения нейронной сети. Модель последовательно обрабатывает обучающие данные (X_train и y_train) в течение 5 эпох (полных проходов по всему набору данных). На каждом шаге данные разбиваются на батчи размером 512 примеров, что позволяет эффективно использовать память и ускорить обучение. В процессе обучения модель выполняет прямое распространение (forward pass), вычисляет ошибку (loss) с помощью функции потерь binary_crossentropy, а затем обновляет свои веса с помощью оптимизатора Adam, минимизируя ошибку. После каждой эпохи модель оценивается на валидационных данных (X_test и y_test), что позволяет отслеживать её обобщающую способность и избегать переобучения. Результаты обучения (ошибка и точность на тренировочных и валидационных данных) сохраняются в объекте history, который можно использовать для анализа и визуализации процесса обучения. Чем больше батчей, тем быстрее процесс обучения, но требуется больше памяти.

Далее была проведена оценка модели.

In [22]:
# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точность на тестовых данных: {accuracy:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8509 - loss: 0.4570
Точность на тестовых данных: 0.85


Команда model.evaluate используется для оценки обученной модели на тестовых данных (X_test и y_test). В процессе выполнения модель обрабатывает тестовые данные, вычисляет значение функции потерь (loss) и точность (accuracy), которые показывают, насколько хорошо модель справляется с предсказаниями на новых, ранее не виденных данных. В данном случае выводится точность модели на тестовых данных, округлённая до двух знаков после запятой. Например, если точность составляет 0.85, это означает, что модель правильно классифицировала 85% тестовых примеров. Этот этап позволяет оценить качество модели и её способность обобщать знания на реальных данных.